In [1]:
import numpy as np
import json
import pandas as pd
import os, glob, re, shutil
import PIL.ExifTags as ExifTags
import PIL.Image as Image
from PIL import ImageOps
import numpy as np
from tqdm import tqdm
from typing import Union
# import cv2

In [3]:
# same with .npy file just no "data" key
file_path = '/cs/student/projects4/ml/2023/asrivast/datasets/bicycle/poses_bounds.npy'
# load the .npy data
data = np.load(file_path)
print(data.shape)

for i in range(0, 10):
    print(data[i])

(194, 17)
[-3.80442207e-02  6.35684066e-01 -7.71011287e-01 -3.00268179e+00
  3.28600000e+03  9.83836581e-01  1.58935283e-01  8.24933848e-02
  1.40077264e+00  4.94600000e+03  1.74980627e-01 -7.55410712e-01
 -6.31455807e-01 -2.22840056e+00  4.64950598e+03  3.46747820e+00
  5.80139520e+01]
[ 2.25899129e-01  6.09137784e-01 -7.60210986e-01 -3.04942107e+00
  3.28600000e+03  9.04358729e-01  1.58927312e-01  3.96077517e-01
  1.68893054e+00  4.94600000e+03  3.62084069e-01 -7.76977007e-01
 -5.14977531e-01 -2.21220869e+00  4.64950598e+03  3.63105887e+00
  6.10263076e+01]
[ 1.48151567e-02  4.39289825e-01 -8.98223224e-01 -3.51985545e+00
  3.28600000e+03  9.60731350e-01  2.42684359e-01  1.34534659e-01
  1.31549645e+00  4.94600000e+03  2.77084434e-01 -8.64944363e-01
 -4.18444100e-01 -1.36722490e+00  4.64950598e+03  3.66833755e+00
  5.63840067e+01]
[ 3.45445768e-01  4.70172859e-01 -8.12160516e-01 -3.59115626e+00
  3.28600000e+03  8.53859893e-01  2.01583967e-01  4.79882473e-01
  1.49892987e+00  4.946000

# Reading the .ply file from COLMAP and converting it to .npz

format of numpy array to be stored in .npz file
* (N, 7) where N is total number of points
* 0-2 x,y,z
* 3-5 R,G,B
* 7 all ones

In [12]:
from plyfile import PlyData, PlyElement 

ply_file= "/cs/student/projects4/ml/2023/asrivast/datasets/portsmouth_video/frames/sparse_000001/0/points3D.ply"
with open(ply_file, 'rb') as f:
    ply_data: PlyData = PlyData.read(f)
    x = ply_data['vertex']['x']
    y = ply_data['vertex']['y']
    z = ply_data['vertex']['z']
    r = ply_data['vertex']['red']
    g = ply_data['vertex']['green']
    b = ply_data['vertex']['blue']
    seven= np.zeros(x.shape[0], dtype=np.float32)

    # print data types of each
    print(f"x: {x.dtype}, y: {y.dtype}, z: {z.dtype}, r: {r.dtype}, g: {g.dtype}, b: {b.dtype}, seven: {seven.dtype}")

    # need to change the dtype of R G B to float32 as well
    r = r.astype(np.float32)/255
    g = g.astype(np.float32)/255
    b = b.astype(np.float32)/255

    print(f"x: {x.dtype}, y: {y.dtype}, z: {z.dtype}, r: {r.dtype}, g: {g.dtype}, b: {b.dtype}, seven: {seven.dtype}")
    # stack them together
    points = np.vstack((x, y, z, r, g, b, seven)).T
    
    # now save it as a .npz file with a key "data"
    npz_file= "init_pt_cld_ours.npz"
    npz= {}
    np.savez_compressed(npz_file, data=points)
    print(f"Saved {npz_file}")


x: float32, y: float32, z: float32, r: uint8, g: uint8, b: uint8, seven: float32
x: float32, y: float32, z: float32, r: float32, g: float32, b: float32, seven: float32
Saved init_pt_cld_ours.npz


# Reading the .npz file and converting it to .ply file

In [13]:
import numpy as np
from plyfile import PlyData, PlyElement

# Assuming 'points' is your numpy array of shape (N, 7)
# Load or define your numpy array here if needed
file_path = 'init_pt_cld_ours.npz'

# Load the .npz file
data = np.load(file_path)
print(data['data'].shape,)
print(data['data'][:,0].dtype, data['data'][:,3].dtype, data['data'][:,6].dtype)
points = data['data']

# Extract the first 6 columns for x, y, z, R, G, B
vertices_data = points[:, :6]

# # Convert to a list of tuples, each tuple representing a vertex
vertices_list = [tuple(vertex) for vertex in vertices_data]
# scale and convert to uint all RGB values
RGB_SCALE = 1 if np.max(vertices_data[:,3:6]) > 1 else 255
print(f"RGB_SCALE: {RGB_SCALE}")
vertices_list = [(x, y, z, int(r* RGB_SCALE), int(g* RGB_SCALE), int(b*RGB_SCALE)) for x, y, z, r, g, b in vertices_list]

# # Define the vertex element
vertex_element = PlyElement.describe(np.array(vertices_list, dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4'), ('red', 'u1'), ('green', 'u1'), ('blue', 'u1')]), 'vertex')

# # Create the PlyData object
ply_data = PlyData([vertex_element])

# # Save the .ply file
# ply_file_path = "output.ply"
# ply_data.write(ply_file_path)

# print(f"PLY file saved to {ply_file_path}")

(8304, 7)
float32 float32 float32
RGB_SCALE: 255


In [58]:
zeros= points.shape[0]- np.sum(points[:,6])
ones= np.sum(points[:,6])
print(f"zeros: {zeros}, ones: {ones} Total: {zeros+ones}")

zeros: 58964.0, ones: 135141.0 Total: 194105.0


In [33]:

# Specify the path to the .npz file
file_path = '/cs/student/projects4/ml/2023/asrivast/datasets/dynamic_3DGS/data/portsmouth/init_pt_cld.npz'

# Load the .npz file
data = np.load(file_path)
print(data['data'].shape)

for i in range(0, 1):
    x,y,z = data['data'][i][0:3]
    R,G,B = data['data'][i][3:6]
    # pritn values and their data types
    print(x, type(x))
    print(y, type(y))
    print(z, type(z))
    print(R, type(R), R*255)
    print(G, type(G), G*255)
    print(B, type(B), B*255)


(8304, 7)
0.04604692384600639 <class 'numpy.float64'>
0.634263813495636 <class 'numpy.float64'>
-0.1974332332611084 <class 'numpy.float64'>
238.0 <class 'numpy.float64'> 60690.0
218.0 <class 'numpy.float64'> 55590.0
217.0 <class 'numpy.float64'> 55335.0


# Reading points3D.bin and convert it to .npz

In [6]:
import utils_data_making
import utils_colmap

In [9]:
pt_cld_path = os.path.join("/cs/student/projects4/ml/2023/asrivast/datasets/portsmouth_video/frames/sparse_000001/0/points3D.bin")
xyzs, rgbs, _ = utils_colmap.read_points3D_binary(pt_cld_path)
seg = np.ones_like(xyzs[:, 0])[:, None]   # Always static for now, segmentation always 1
rgbs = rgbs / 255.0
pt_cld = dict()
pt_cld = np.concatenate((xyzs, rgbs, seg), axis=1).tolist()
# if not os.path.exists(os.path.join(".")):
#     os.makedirs(os.path.join(args.output_path, args.dataset_name))
np.savez(os.path.join('init_pt_cld_other.npz'), data=pt_cld)
print('Point cloud saved')

Point cloud saved


# Reading the .bin files from COLMAP and converting it to train_meta.json

## First we see what kind of file is train_meta.json

In [6]:
a= [1,2,3,4,5,6,7]
# get third and fifth element
b= [a[2], a[4]]
b

[3, 5]

In [19]:
meta_json= "/cs/student/projects4/ml/2023/asrivast/datasets/dynamic_3DGS/data/boxes_small/train_meta_org.json"
with open(meta_json, 'r') as f:
    meta_dict= json.load(f)
    print(f"Loaded {meta_json}")
    for key in meta_dict.keys():
        value= np.array(meta_dict[key])
        if value.shape == ():
            print(f"{key}: {value}")
        else:
            print(f"{key}: {value.shape}")

    print("#####################")
    k_np= np.array(meta_dict['k'])
    k_np= k_np[:,(0, -1), :,:]
    print(k_np.shape)

    w2c_np= np.array(meta_dict['w2c'])
    w2c_np= w2c_np[:,(0, -1), :,:]
    print(w2c_np.shape)

    fn_np= np.array(meta_dict['fn'])
    fn_np= fn_np[:,(0, -1)]
    print(fn_np.shape)

    cam_id_np= np.array(meta_dict['cam_id'])
    cam_id_np= cam_id_np[:,(0, -1)]
    print(cam_id_np.shape)

    new_meta_dict= meta_dict.copy()
    new_meta_dict['k']= k_np.tolist()
    new_meta_dict['w2c']= w2c_np.tolist()
    new_meta_dict['fn']= fn_np.tolist()
    new_meta_dict['cam_id']= cam_id_np.tolist()
    with open("test_meta.json", 'w') as f:
        json.dump(new_meta_dict, f, indent=4)
        print(f"Saved test_meta.json")

Loaded /cs/student/projects4/ml/2023/asrivast/datasets/dynamic_3DGS/data/boxes_small/train_meta_org.json
w: 640
h: 360
k: (150, 27, 3, 3)
w2c: (150, 27, 4, 4)
fn: (150, 27)
cam_id: (150, 27)
#####################
(150, 2, 3, 3)
(150, 2, 4, 4)
(150, 2)
(150, 2)
Saved test_meta.json


# from .bin file

In [133]:
import read_write_model

cams= read_write_model.read_cameras_binary("/cs/student/projects4/ml/2023/asrivast/datasets/portsmouth_video/frames/sparse_000001/0/cameras.bin")
print(cams)
print(type(cams))
print(type(cams[1]))
print(cams[1])

{1: Camera(id=1, model='PINHOLE', width=1220, height=994, params=array([1928.96714337, 1920.45748488,  610.        ,  497.        ]))}
<class 'dict'>
<class 'read_write_model.Camera'>
Camera(id=1, model='PINHOLE', width=1220, height=994, params=array([1928.96714337, 1920.45748488,  610.        ,  497.        ]))


In [134]:
images= read_write_model.read_images_binary("/cs/student/projects4/ml/2023/asrivast/datasets/portsmouth_video/frames/sparse_000001/0/images.bin")
# total number of keys= total number of images
cam_id_map = {} # cameran_num : COLMAP index
for COLMAP_index in images.keys():
    camera_num= int(re.split("[-_]", images[COLMAP_index].name)[1])
    cam_id_map[camera_num]= COLMAP_index
    print("COLMAP index", COLMAP_index, images[COLMAP_index].name, "CAM num", camera_num)

COLMAP index 1 cam-005_000001.jpg CAM num 5
COLMAP index 2 cam-002_000001.jpg CAM num 2
COLMAP index 3 cam-006_000001.jpg CAM num 6
COLMAP index 4 cam-007_000001.jpg CAM num 7
COLMAP index 5 cam-001_000001.jpg CAM num 1
COLMAP index 6 cam-003_000001.jpg CAM num 3
COLMAP index 7 cam-008_000001.jpg CAM num 8
COLMAP index 8 cam-011_000001.jpg CAM num 11
COLMAP index 9 cam-010_000001.jpg CAM num 10
COLMAP index 10 cam-009_000001.jpg CAM num 9
COLMAP index 11 cam-004_000001.jpg CAM num 4
COLMAP index 12 cam-012_000001.jpg CAM num 12
COLMAP index 13 cam-013_000001.jpg CAM num 13
COLMAP index 14 cam-014_000001.jpg CAM num 14
COLMAP index 15 cam-015_000001.jpg CAM num 15
COLMAP index 16 cam-021_000001.jpg CAM num 21
COLMAP index 17 cam-022_000001.jpg CAM num 22
COLMAP index 18 cam-016_000001.jpg CAM num 16
COLMAP index 19 cam-017_000001.jpg CAM num 17
COLMAP index 20 cam-018_000001.jpg CAM num 18
COLMAP index 21 cam-019_000001.jpg CAM num 19
COLMAP index 22 cam-020_000001.jpg CAM num 20
COLMAP

In [147]:
train_cams= [i for i in range(1, 33)] # all cameras
train_cams= [cam_id_map[cam] for cam in train_cams]


test_cams= [2,15,25,30] # cam 2 and 19
test_cams= [cam_id_map[cam] for cam in test_cams]

# remove test cams from train cams
train_cams= [cam for cam in train_cams if cam not in test_cams]


k_np= np.zeros((len(test_cams),3,3))
w2c_np= np.zeros((len(test_cams),4,4))
fn = []
cam_ids_np= np.zeros(len(test_cams), dtype=np.uint)


for i, image_id in enumerate(test_cams):
    
    img_name= images[image_id].name
    cam_obj= cams[images[image_id].camera_id]

    q_vec= images[image_id].qvec
    t_vec= images[image_id].tvec

    w= cam_obj.width
    h= cam_obj.height

    fx= cam_obj.params[0]
    fy= cam_obj.params[1]
    cx= cam_obj.params[2]
    cy= cam_obj.params[3]


    k= np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
    w2c= np.eye(4)
    w2c[:3,:3]= read_write_model.qvec2rotmat(q_vec)
    w2c[:3,3]= t_vec

    k_np[i]= k
    w2c_np[i]= w2c
    

    fn.append(img_name)
    cam_ids_np[i]= image_id

# for image_obj in images:
#     print(type(images[image_obj]))

In [148]:
# print shapes of all arrays to which we added in previous steps
print("k ", k_np.shape)
print("w2c ", w2c_np.shape)
print("fn ", len(fn))
print("cam_id", cam_ids_np.shape)

# print

k  (4, 3, 3)
w2c  (4, 4, 4)
fn  4
cam_id (4,)


In [149]:
fn

['cam-002_000001.jpg',
 'cam-015_000001.jpg',
 'cam-025_000001.jpg',
 'cam-030_000001.jpg']

In [150]:
# now add a dimension in beggining in each of them and repead them total_frames times
total_frames= 199
k_np_full = np.repeat(k_np[np.newaxis, ...], total_frames, axis=0)
w2c_np_full = np.repeat(w2c_np[np.newaxis, ...], total_frames, axis=0)


fn_full= []
for i in range(1, total_frames+1):
    frame_num= str(i).zfill(6)
    fn_new= fn.copy()
    for i,_ in enumerate(fn_new):
        cam_num= int(re.split("[-_]", _)[1])
        fn_new[i]= fn_new[i].replace("000001", frame_num)
        fn_new[i]= f"{cam_num}/{fn_new[i][8:-4]}.JPG"

        
    fn_full.append(fn_new)

cam_ids_np_full= np.repeat(cam_ids_np[np.newaxis, ...], total_frames, axis=0)

In [151]:
# print shapes along with names agains
print("k ", k_np_full.shape)
print("w2c ", w2c_np_full.shape)
print("fn ", np.array(fn_full).shape)
print("cam_id", cam_ids_np_full.shape)


k  (199, 4, 3, 3)
w2c  (199, 4, 4, 4)
fn  (199, 4)
cam_id (199, 4)


In [152]:
fn_full[-1]

['2/000199.JPG', '15/000199.JPG', '25/000199.JPG', '30/000199.JPG']

In [153]:
meta_dict= {"w": 1220,
            "h": 994,
            "k": k_np_full.tolist(),
            "w2c": w2c_np_full.tolist(),
            "fn": fn_full,
            "cam_id": cam_ids_np_full.tolist()}

json_path= "/cs/student/projects4/ml/2023/asrivast/datasets/dynamic_3DGS/data/portsmouth/test_meta.json"
with open(json_path, 'w') as meta_json:
    json.dump(meta_dict, meta_json, indent=4)
    print(f"Saved {json_path}")


Saved /cs/student/projects4/ml/2023/asrivast/datasets/dynamic_3DGS/data/portsmouth/test_meta.json


# transform.json from colmap2nerf.py

In [8]:
meta_json= "/cs/student/projects4/ml/2023/asrivast/Dynamic3DGaussians/transforms.json"
with open(meta_json, 'r') as f:
    meta_dict= json.load(f)
    print(f"Loaded {meta_json}")
    for key in meta_dict.keys():
        value= np.array(meta_dict[key])
        if value.shape == ():
            print(f"{key}: {value}")
        else:
            print(f"{key}: {value.shape}")

Loaded /cs/student/projects4/ml/2023/asrivast/Dynamic3DGaussians/transforms.json
camera_angle_x: 0.612561364141283
camera_angle_y: 0.5064733756797346
fl_x: 1928.9671433700703
fl_y: 1920.4574848758382
k1: 0
k2: 0
k3: 0
k4: 0
p1: 0
p2: 0
is_fisheye: False
cx: 610.0
cy: 497.0
w: 1220.0
h: 994.0
aabb_scale: 32
frames: (32,)


In [112]:
bottom = np.array([0.0, 0.0, 0.0, 1.0]).reshape([1, 4])
m = np.concatenate([np.concatenate([rot_mat, t_vec], 1), bottom], 0)
c2w = np.linalg.inv(m)
print(c2w)
# cam2world= (-rot_mat.T)*(t_vec)
# print(cam2world)

[[-0.71022167  0.4768192  -0.51790793  2.38413646]
 [ 0.13507749  0.81431994  0.56447951 -1.50049363]
 [ 0.69089742  0.33094788 -0.64275521  2.69671221]
 [ 0.          0.          0.          1.        ]]


# colmap2nerf.py detectron2

In [4]:
from collections import namedtuple
import detectron2
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor


In [5]:

SCRIPTS_FOLDER = "."
# make args as a named tuple
CLI_args = namedtuple("CLI_args", "mask_categories")  # see category2id.json for all categories
args= CLI_args(mask_categories= ["person"])
args.mask_categories

['person']

In [6]:

category2id = json.load(open(os.path.join(SCRIPTS_FOLDER, "category2id.json"), "r"))
mask_ids = [category2id[c] for c in args.mask_categories]

cfg = get_cfg()
# Add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo.
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)


In [41]:
all_jpgs= glob.glob("/cs/student/projects4/ml/2023/asrivast/datasets/dynamic_3DGS/data/portsmouth/ims/**/*.JPG", recursive=True)
print(f"Total number of images: {len(all_jpgs)} per camera {len(all_jpgs)//32}")

Total number of images: 6368 per camera 199


In [46]:

for frame in tqdm(all_jpgs):
	# print(frame["file_path"])
	# check if the image is present
	# print(os.path.exists(frame["file_path"]))
	img = cv2.imread(frame)
	# print(img.shape)
	outputs = predictor(img)

	output_mask = np.zeros((img.shape[0], img.shape[1]))
	for i in range(len(outputs["instances"])):
		if outputs["instances"][i].pred_classes.cpu().numpy()[0] in mask_ids:
			pred_mask = outputs["instances"][i].pred_masks.cpu().numpy()[0]
			output_mask = np.logical_or(output_mask, pred_mask)

	rgb_path = os.path.join(frame)
	# # print(rgb_path)
	mask_name = rgb_path.replace("ims", "seg").replace(".JPG", ".png")
	# print(mask_name)
	# print(output_mask.shape)

	if not os.path.exists(os.path.dirname(mask_name)):
		os.makedirs(os.path.dirname(mask_name), exist_ok=True)
	cv2.imwrite(mask_name, (output_mask*255).astype(np.uint8))
	# print(mask_name)
	# print(os.path.exists(mask_name))

	# break

100%|██████████| 6368/6368 [05:24<00:00, 19.65it/s]


# Reading the output .npz file

In [65]:
npz_file= "/cs/student/projects4/ml/2023/asrivast/datasets/dynamic_3DGS/output/exp3/boxes/params.npz"
with np.load(npz_file) as data:
    for file in data.files:
        print(f"{file}: {data[file].shape}")

        if file == "cam_m" or file == "cam_c":
            print(data[file])

    

means3D: (11, 355146, 3)
rgb_colors: (11, 355146, 3)
seg_colors: (355146, 3)
unnorm_rotations: (11, 355146, 4)
logit_opacities: (355146, 1)
log_scales: (355146, 3)
cam_m: (50, 3)
[[ 0.04052375  0.04089984  0.03511785]
 [-0.10412157 -0.11086568 -0.11600678]
 [ 0.03189565  0.04502699  0.03034953]
 [-0.06717889 -0.07117973 -0.08319283]
 [-0.00327357  0.01511976  0.0076265 ]
 [ 0.09751539  0.09298394  0.10647158]
 [ 0.05988395  0.03975919  0.05563907]
 [ 0.00337028  0.01964025  0.00806313]
 [-0.00923424  0.01268488  0.0058958 ]
 [-0.11130295 -0.11080908 -0.11390278]
 [-0.01869849 -0.02034263 -0.01551975]
 [-0.05780772 -0.06925236 -0.06072897]
 [-0.01703494 -0.00727891 -0.01703449]
 [-0.0618599  -0.06228618 -0.06967499]
 [ 0.02166728  0.01342456  0.01806911]
 [-0.02762344 -0.01596763 -0.02078595]
 [ 0.06977711  0.07378475  0.07342494]
 [ 0.00482329  0.03206026  0.01437046]
 [-0.06484304 -0.0602063  -0.05086702]
 [-0.01094176  0.00574239 -0.00309625]
 [-0.01958307 -0.01359787 -0.03105757]
 [

# Move first frame to a new folder

In [22]:
import pandas as pd
import os, glob, re, shutil
import PIL.ExifTags as ExifTags
import PIL.Image as Image
from PIL import ImageOps
import numpy as np
from tqdm import tqdm
from typing import Union
import piexif

In [27]:
all_jpgs= glob.glob("/cs/student/projects4/ml/2023/asrivast/datasets/dynamic_3DGS/data/basketball/ims/*/000000.jpg")
print(len(all_jpgs))

new_folder= "/cs/student/projects4/ml/2023/asrivast/datasets/dynamic_3DGS/data/basketball/frame_000000"
os.makedirs(new_folder, exist_ok=True)
for jpg in tqdm(all_jpgs):
    cam_num= os.path.basename(os.path.dirname(jpg))
    frame0_file= os.path.join(new_folder, f"cam-{cam_num}_000000.jpg")
    shutil.copy2(jpg, frame0_file)

31


100%|██████████| 31/31 [00:00<00:00, 527.43it/s]
